In [1]:
pip install python-dotenv



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
api_key = os.getenv('API_KEY')
print(api_key)

24DD4E29-B6FF-31DC-9066-43A4107B0C42


In [3]:
import requests
import pandas as pd
base_url = 'https://quickstats.nass.usda.gov/api/api_GET/'

def fetch_data(year):
    # Set up the parameters for the API request
    
    groups = ['FIELD CROPS', 'FRUIT & TREE NUTS', 'VEGETABLES']
    states = ['CA', 'CO', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'ME', 'MD', 'MI',
       'MN', 'MO', 'MT', 'NE', 'NV', 'NM', 'NY', 'NC', 'ND', 'OH', 'OR',
       'PA', 'SD', 'UT', 'VA', 'WA', 'WI', 'WY', 'VT', 'MA', 'TX', 'AZ',
       'AR', 'DE', 'FL', 'GA', 'LA', 'NJ', 'OK', 'RI', 'SC', 'TN', 'WV',
       'CT', 'MS', 'AL', 'HI', 'NH', 'AK']
    total_df = pd.DataFrame()
    
    for state in states:
            field_crop_params = {
                'key': api_key,
                'source_desc': 'CENSUS',
                'sector_desc': 'CROPS',
                'statisticcat_desc': 'AREA HARVESTED',
                'state_alpha':state,
                # 'group_desc': group,
                'year': year,
                'agg_level_desc': 'STATE',
                'freq_desc': 'Annual',
                'format': 'JSON',  # Desired response format
                'unit_desc': 'ACRES'
            }
            # Send the GET request
            response = requests.get(base_url, params=field_crop_params)
            # Check if the request was successful
            if response.status_code == 200:
                raw = response.json()  # Parse the JSON response
                
                df = pd.DataFrame(raw['data'])
                total_df = pd.concat([df, total_df], ignore_index=True)
            else:
                print(state)
                print(f"Error fetching data {response.status_code}")
                print("HUH")
                return None

    return total_df
    
    
#stdrvmkcr


In [4]:
def normalize_data(data):
    total_df = data
    filtered_df = total_df[total_df['short_desc'].str.endswith('ACRES HARVESTED')]
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('FRUIT TOTALS', 'FRUIT')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('FRUIT, OTHER', 'FRUIT')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('TREE NUT TOTALS', 'TREE NUT')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('TREE NUTS, OTHER', 'TREE NUT')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('BERRIES, OTHER', 'BERRIES')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('BERRY TOTALS', 'BERRIES')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('FIELD CROP', 'FIELD CROPS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('FIELD CROP TOTALS', 'FIELD CROPS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('FIELD CROPS, OTHER', 'FIELD CROPS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('CITRUS TOTALS', 'CITRUS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('TREE NUT TOTALS', 'TREE NUTS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('TREE NUTS, OTHER', 'TREE NUTS')  

    filtered_df['Value'] = filtered_df['Value'].replace({'(D)': np.nan}, regex=True)  # Replace (D) with NaN
    filtered_df['Value'] = filtered_df['Value'].str.replace(',', '')  # Remove commas
    filtered_df['Value'] = pd.to_numeric(filtered_df['Value'], errors='coerce')  # Convert to numeric, invalid parsing will become NaN
    agreggate = filtered_df.copy()
    agreggate = pd.DataFrame(agreggate)
    
    def normalize_commodity(row):
    # Define a list of commodities to normalize
        commodities_to_normalize =agreggate['commodity_desc'].unique()
        
        # Loop through each commodity and check if it exists in the row's 'Commodity' column
        for commodity in commodities_to_normalize:
            if commodity in row['commodity_desc']:
                return commodity
        
        return row['commodity_desc']
    # Apply the normalization function to the 'Commodity' column
    agreggate['commodity_desc'] = agreggate.apply(normalize_commodity, axis=1)
    # Group by relevant columns and sum the values
    aggregated_df = (
        agreggate.groupby(['state_name', 'commodity_desc'], as_index=False)
        .agg({'Value': 'sum'})
    )

    # Pivot the table to have commodities as columns
    pivot_df = aggregated_df.pivot(index='state_name', columns='commodity_desc', values='Value').reset_index()

    pivot_df = pivot_df.fillna(0)

    return pivot_df
    

In [12]:
for i in range(2024, 2026):
    data = fetch_data(i)
    if data is not None and not data.empty:
        print(f"{i} sucess")
        agreggated = normalize_data(data)
        agreggated.to_csv(f"{i}_agreggated_api")
    else:
        print(f'{i} failure')

CA
Error fetching data 400
HUH
2024 failure
CA
Error fetching data 400
HUH
2025 failure
